Extracted data from the website "https://www.veritree.com"

In [4]:
import json
import pandas as pd
with open('../input/Verritree/verritree1.json', 'r') as file:
    data = json.load(file)

print(type(data))


print(data[0])


if isinstance(data, list) and isinstance(data[0], dict):
    df1 = pd.DataFrame(data)
    df1.info()

<class 'list'>
{'id': 22, 'public_id': 'tambohibe', 'planting_site_id': 2, 'country_id': 0, 'organization_id': 1, 'subsite_type_id': 1, 'plus_code': '5HP83C54+47', 'name': 'Tambohibe', 'description': 'The Betsiboka estuary is a highly dynamic forest landscape with seasonal flooding depositing large amounts of silt and soil from inland. Dominated, by mangrove species such as Ceriops tagal, Rhizophora mucronata, and Avicennia marina, the area represents an example of globally important mangrove forest. Mangrove forests are well known for their exceptional value in terms of ecosystem services. Apart of carbon cycling, mangroves also provide significant silt filtration, fish nurseries, and protection against storm surges.', 'legacy': 0, 'created_at': '2020-09-02T00:57:57.000000Z', 'updated_at': '2023-01-05T00:28:36.000000Z', 'forest_type_id': 1, 'rainfall_yearly_mm': 432, 'enabled': 1, 'media': [], 'organization_name': 'Eden Reforestation Projects', 'standards': [{'theme_id': 247, 'categor

In [13]:
df1.head()

,id,public_id,planting_site_id,country_id,organization_id,subsite_type_id,plus_code,name,description,legacy,...,planting_site.visibility,planting_site.name,planting_site.country.id,planting_site.country.full_name,planting_site.country.name,socioeconomic_impacts,additional_info_description,additional_info_title,planting_site.description,site_code
0,22,tambohibe,2,0,1,1,5HP83C54+47,Tambohibe,The Betsiboka estuary is a highly dynamic fore...,0,...,0,Bestiboka Estuary,450,Republic of Madagascar,Madagascar,NaN,NaN,NaN,NaN,NaN
1,25,tanindraza-2,2,0,1,3,5HP84933+V6,Tanindraza 2,The Betsiboka estuary is a highly dynamic fore...,0,...,0,Bestiboka Estuary,450,Republic of Madagascar,Madagascar,null,null,null,NaN,NaN
2,29,madjuva,8,0,1,1,5G5JPMFR+W5,Madjuva,Madjuva is a critical mangrove habitat that pr...,0,...,0,Maputo Bay,508,Republic of Mozambique,Mozambique,"Through this project, Eden offers steady emplo...",Based on careful observation and field studies...,Survivability Report,NaN,NaN
3,30,nawalparassi,9,0,1,2,7MV6R249+2W,Nawalparassi,test,0,...,0,Ruchang,524,Nepal,Nepal,null,null,null,NaN,NaN
4,32,antsanitia-dry-deciduous-1,10,0,1,1,5HP8CC36+HM,Antsanitia Dry Deciduous 1,The Antsanitia Dry Deciduous site is situated ...,1,...,0,Antsanitia,450,Republic of Madagascar,Madagascar,null,null,Survivability Update,NaN,NaN


In [20]:
df1["area_square_kilometers"] = df1["area_square_meters"] / 1_000_000

In [22]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'planting_site_id': 'site_id_reported',
    'id': 'project_id_reported',
    'geofence.coordinates': 'geometry_reported',
    'additional_info_description': 'description_reported',
    'planting_site.country.name': 'country',
    
    'forest_type.average_survival_rate': 'survival_rate_reported',
    'species': 'species_planted_reported',
    "area_square_kilometers":"site_sqkm"
}

df1 = df1[list(columns_rename_mapping.keys())]


df1.rename(columns=columns_rename_mapping, inplace=True)

/var/folders/_l/gj90rjl90fngb4_8ph5x5jhc0000gn/T/ipykernel_3241/1372476715.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns=columns_rename_mapping, inplace=True)


In [23]:
df1 = df1.assign(species_count_reported=None,trees_planted_reported=None,planting_date_reported=None)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   site_id_reported          50 non-null     int64  
 1   project_id_reported       50 non-null     int64  
 2   geometry_reported         50 non-null     object 
 3   description_reported      5 non-null      object 
 4   country                   50 non-null     object 
 5   survival_rate_reported    50 non-null     int64  
 6   species_planted_reported  50 non-null     object 
 7   site_sqkm                 50 non-null     float64
 8   species_count_reported    0 non-null      object 
 9   trees_planted_reported    0 non-null      object 
 10  planting_date_reported    0 non-null      object 
dtypes: float64(1), int64(3), object(7)
memory usage: 4.4+ KB


In [5]:
with open('../input/Verritree/verritree2.json', 'r') as file:
    data = json.load(file)

print(type(data))


print(data[0])


if isinstance(data, list) and isinstance(data[0], dict):
    df2 = pd.DataFrame(data)
    df2.info()

<class 'list'>
{'id': 136, 'public_id': 'vilcanota-vilcanota-per', 'planting_site_id': 68, 'country_id': 0, 'organization_id': 48, 'subsite_type_id': 25, 'plus_code': '57R9XV2J+M5', 'name': 'Vilcanota', 'description': '', 'legacy': 0, 'created_at': '2023-06-15T16:26:57.000000Z', 'updated_at': '2023-06-15T16:26:58.000000Z', 'forest_type_id': 51, 'rainfall_yearly_mm': 936, 'enabled': 1, 'media': [], 'organization_name': 'Global Forest Generation', 'standards': [{'theme_id': 205, 'category': 'design', 'theme_name': 'Project Outline', 'goal_id': 210, 'goal_name': 'Analysis', 'indicator_id': 211, 'indicator_name': 'Sample Plots', 'is_planting_counter': 0}, {'theme_id': 205, 'category': 'design', 'theme_name': 'Project Outline', 'goal_id': 206, 'goal_name': 'Governance', 'indicator_id': 209, 'indicator_name': 'DELETE', 'is_planting_counter': 0}, {'theme_id': 205, 'category': 'design', 'theme_name': 'Project Outline', 'goal_id': 206, 'goal_name': 'Governance', 'indicator_id': 207, 'indicator_

In [27]:
df2["area_square_kilometers"] = df2["area_square_meters"] / 1_000_000

In [32]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'planting_site_id': 'site_id_reported',
    'id': 'project_id_reported',
    'geofence.coordinates': 'geometry_reported'
    ,'planting_site.country.name': 'country',
    "planting_site.description":"description_reported",
    'forest_type.average_survival_rate': 'survival_rate_reported',
    'species': 'species_planted_reported',
    "area_square_kilometers":"site_sqkm"
    
}

df2 = df2[list(columns_rename_mapping.keys())]


df2.rename(columns=columns_rename_mapping, inplace=True)

/var/folders/_l/gj90rjl90fngb4_8ph5x5jhc0000gn/T/ipykernel_3241/2170220975.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.rename(columns=columns_rename_mapping, inplace=True)


In [33]:
df2 = df2.assign(species_count_reported=None,trees_planted_reported=None,planting_date_reported=None)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   site_id_reported          50 non-null     int64  
 1   project_id_reported       50 non-null     int64  
 2   geometry_reported         50 non-null     object 
 3   country                   50 non-null     object 
 4   description_reported      2 non-null      object 
 5   survival_rate_reported    50 non-null     int64  
 6   species_planted_reported  50 non-null     object 
 7   site_sqkm                 50 non-null     float64
 8   species_count_reported    0 non-null      object 
 9   trees_planted_reported    0 non-null      object 
 10  planting_date_reported    0 non-null      object 
dtypes: float64(1), int64(3), object(7)
memory usage: 4.4+ KB


In [6]:
with open('../input/Verritree/verritree3.json', 'r') as file:
    data = json.load(file)

print(type(data))


print(data[0])


if isinstance(data, list) and isinstance(data[0], dict):
    df3 = pd.DataFrame(data)
    df3.info()

<class 'list'>
{'id': 219, 'public_id': 'marremissa-mozambique-moz', 'planting_site_id': 97, 'country_id': 0, 'organization_id': 1, 'subsite_type_id': 1, 'plus_code': '5GJRJMMF+G2', 'name': 'Marremissa', 'description': '', 'legacy': 1, 'created_at': '2024-03-28T04:53:03.000000Z', 'updated_at': '2024-03-28T04:53:03.000000Z', 'forest_type_id': 93, 'enabled': 1, 'media': [], 'organization_name': 'Eden Reforestation Projects', 'standards': [{'theme_id': 205, 'category': 'design', 'theme_name': 'Project Outline', 'goal_id': 210, 'goal_name': 'Analysis', 'indicator_id': 211, 'indicator_name': 'Sample Plots', 'is_planting_counter': 0}, {'theme_id': 205, 'category': 'design', 'theme_name': 'Project Outline', 'goal_id': 206, 'goal_name': 'Governance', 'indicator_id': 209, 'indicator_name': 'DELETE', 'is_planting_counter': 0}, {'theme_id': 205, 'category': 'design', 'theme_name': 'Project Outline', 'goal_id': 206, 'goal_name': 'Governance', 'indicator_id': 207, 'indicator_name': 'Documentation',

In [36]:
df3["area_square_kilometers"] = df3["area_square_meters"] / 1_000_000

In [39]:
# Renaming the columns to follow our naming format in the paper columns section
columns_rename_mapping = {
    'planting_site_id': 'site_id_reported',
    'id': 'project_id_reported',
    'geofence.coordinates': 'geometry_reported'
    ,'planting_site.country.name': 'country',
    "forest_type.description":"description_reported",
    'forest_type.average_survival_rate': 'survival_rate_reported',
    'species': 'species_planted_reported',
    "area_square_kilometers":"site_sqkm"
    
}

df3 = df3[list(columns_rename_mapping.keys())]


df3.rename(columns=columns_rename_mapping, inplace=True)

/var/folders/_l/gj90rjl90fngb4_8ph5x5jhc0000gn/T/ipykernel_3241/2848268690.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.rename(columns=columns_rename_mapping, inplace=True)


In [40]:
df3 = df3.assign(species_count_reported=None,trees_planted_reported=None,planting_date_reported=None)
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   site_id_reported          22 non-null     int64  
 1   project_id_reported       22 non-null     int64  
 2   geometry_reported         22 non-null     object 
 3   country                   22 non-null     object 
 4   description_reported      11 non-null     object 
 5   survival_rate_reported    22 non-null     int64  
 6   species_planted_reported  22 non-null     object 
 7   site_sqkm                 22 non-null     float64
 8   species_count_reported    0 non-null      object 
 9   trees_planted_reported    0 non-null      object 
 10  planting_date_reported    0 non-null      object 
dtypes: float64(1), int64(3), object(7)
memory usage: 2.0+ KB


In [41]:

combined_vdf = pd.concat([df1, df2, df3])
combined_vdf.info()


<class 'pandas.core.frame.DataFrame'>
Index: 122 entries, 0 to 21
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   site_id_reported          122 non-null    int64  
 1   project_id_reported       122 non-null    int64  
 2   geometry_reported         122 non-null    object 
 3   description_reported      18 non-null     object 
 4   country                   122 non-null    object 
 5   survival_rate_reported    122 non-null    int64  
 6   species_planted_reported  122 non-null    object 
 7   site_sqkm                 122 non-null    float64
 8   species_count_reported    0 non-null      object 
 9   trees_planted_reported    0 non-null      object 
 10  planting_date_reported    0 non-null      object 
dtypes: float64(1), int64(3), object(7)
memory usage: 11.4+ KB


In [42]:
combined_vdf["website_reported"] = "https://www.veritree.com"
combined_vdf['host_name'] = "Veritree"
combined_vdf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122 entries, 0 to 21
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   site_id_reported          122 non-null    int64  
 1   project_id_reported       122 non-null    int64  
 2   geometry_reported         122 non-null    object 
 3   description_reported      18 non-null     object 
 4   country                   122 non-null    object 
 5   survival_rate_reported    122 non-null    int64  
 6   species_planted_reported  122 non-null    object 
 7   site_sqkm                 122 non-null    float64
 8   species_count_reported    0 non-null      object 
 9   trees_planted_reported    0 non-null      object 
 10  planting_date_reported    0 non-null      object 
 11  website_reported          122 non-null    object 
 12  host_name                 122 non-null    object 
dtypes: float64(1), int64(3), object(9)
memory usage: 13.3+ KB


In [ ]:
combined_vdf.to_csv('../input/verritree/verritree.csv', index=False)